# ***Projet 10 - DETECTION DE FAUX BILLETS - PROGRAMME DETECTION***


---
## Partie 1 - Importation librairies

### 1. Import des librairies et run des fonctions (ok)
---

In [1]:
# Petite fonction pour ne pas afficher les FutureWarning (adaptable selon les warning que l'on veux ou non)
import warnings
warnings.filterwarnings(action = 'ignore', category=FutureWarning)

# Lancement de mon book avec mes fonctions perso
# Importe également les librairies nécessaires (une sorte de boite à outil)
# %run DsFx_fct.ipynb
%run GESSER_David_2_DsFx_fct_102023.ipynb
fct_test()
fct_load()

Librairies utilisées :
+-------------------------------------+
    Pandas : 2.0.3
     Numpy : 1.24.3
Matplotlib : 3.7.2
   Seaborn : 0.12.2
     Spicy : 1.11.1
Statmodels : 0.14.0
   Sklearn : 1.3.0
    JobLib : 1.2.0
+-------------------------------------+


+--o=      Test run DsFx_fct      =o--+

+-------------------------------------+
|   Import lib : OK - Loaded          |
|      Run fct : OK - Loaded          |
+-------------------------------------+
                                       
_______________________________________
 _    __    _                          
| \ _|_   _|___|_  o  | _  _  _| _  _| 
|_/_>|><___|(_ |_  o  |(_)(_|(_|(/_(_| 
 _                                 ... 
|_) _  _  _|\/ _|_ _      _ _      ||| 
| \(/_(_|(_|/   |_(_) |_|_>(/_     000 
_______________________________________


---
## Partie 2 - Fonction de détection

### 2. Détection
---

In [2]:
# Données d'entrées

# Indiquer le nom du .csv contenant les billets à tester :
df = 'billets_test_02.csv'

df_in = pd.read_csv(df, sep = ',') # Vérifier le séparateur utilisé dans le fichier .csv
# Visu des billets à analyser
print("Données d'entrée :")
print('---------------------------------------------------------------------')
df_in.head()

Données d'entrée :
---------------------------------------------------------------------


,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [8]:
# Fonction de détection de faux billtes

def fct_pred_rl(data_in):
    # Préparation des données d'entrées et application du modèle
    # Pour rappel : 0 = Faux - 1 = Vrai (c'est avec cela quele model_02 a été entrainé)
    
    # On récupère nos données d'entrées et on set un index
    data = data_in.set_index(data_in['id']).copy()
    
    # On sait que pour ce modèle nous n'avons pas besoin de ces features (observé lors de la préparation du modèle
    data.drop(['id', 'diagonal', 'height_left'], axis=1, inplace=True)

    # On scale nos données
    data = fct_data_scaler(data, scaler_all = True, method = 'standard')
    # On charge le model_02 - Regression logistique entrainée sur nos données fournies
    model = joblib.load('model_02.joblib')

    # On réalise la prédiction sur les billets fournis avec le model pré-entrainer
    prediction = model.predict(data)
    # display(prediction)
    
    # Probabilité des résultats en %
    proba = pd.DataFrame(model.predict_proba(data)).rename(columns={0: '%_False', 1: '%_True'})
    proba = round(proba*100, 4)
    # On cale l'index
    proba.set_index(data.index, inplace = True)
    # display(proba)
    
    # Préparation du résultat de sortie :
    
    # On créé une colonne dans notre table en ajoutant les labels calculé par le model
    data['pred'] = prediction
    # On ajoute les probabilités des prédictions
    data = pd.merge(left = data, right = proba, left_index = True, right_index = True)
    # On nettoie le df de sortie
    df_out = data.drop(['height_right', 'margin_low', 'margin_up', 'length'], axis = 1)
    df_out['pred'] = df_out['pred'].map({1 : 'True', 0 : 'False'})
    
    # Résultats après application du modèle
    display(df_out)
    
    # Modèle utilisé
    print('+--------------------------+')
    print('')
    print('Modèle utilisé :', model)
    print('Basé sur les features :', model.feature_names_in_)

In [9]:
# Appel de la fonction de détection et réponse
print('+--------------------------+')
print('|    Résultats obtenus     |')
print('+--------------------------+')
fct_pred_rl(df_in)

+--------------------------+
|    Résultats obtenus     |
+--------------------------+


,pred,%_False,%_True
id,,,
B_1,True,0.0033,99.9967
B_2,False,88.7294,11.2706
B_3,True,0.0024,99.9976
B_4,False,99.9888,0.0112
B_5,False,56.3882,43.6118


+--------------------------+

Modèle utilisé : LogisticRegression()
Basé sur les features : ['height_right' 'margin_low' 'margin_up' 'length']


## Test KMeans (pour comparer)

## Test KNN (pour comparer)

In [5]:
# Coding by : David GESSER
# 28/12/2023